<a href="https://colab.research.google.com/github/aleks-haksly/KarpovCources_Hard_DA_advanced/blob/main/03%20-%20Advanced%20experiments%20design/08_stratification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from scipy.stats import norm, t

In [ ]:
!wget https://raw.githubusercontent.com/aleks-haksly/AB_advanced/main/Part12%20-STRATIFICATION/Data/stratification_homework_2.csv -q

In [ ]:
data = pd.read_csv('stratification_homework_2.csv?token=GHSAT0AAAAAACTJMDQNE5T5KNBW5IUWZVZMZULTNQA')
data.head()

,ARPU,is_capital,is_pro,group
0,250.5,region,1,1
1,182.0,capital,0,1
2,75.0,region,0,0
3,532.5,capital,1,0
4,88.0,region,0,1


## Задание 1 ##
Проведите базовый t-test и сделайте вывод по его результатам.

Введите значение p-value в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.

In [ ]:
gr_0 = data.query('group == 0').ARPU
gr_1 = data.query('group == 1').ARPU
print("Базовый p-value для H0 о равенстве средних в группах 1 и 2 равен", ttest_ind(gr_0, gr_1).pvalue.round(3))

Базовый p-value для H0 о равенстве средних в группах 1 и 2 равен 0.065


## Задание 2 (1/3) ##
Найдите наилучшую для разбиения страту.

Рассчитайте значение стратифицированной дисперсии для страты is_capital

In [ ]:
# посчитаем дисперсию и долю в каждой группы при стратификации по ковариате is_capital
data_stratified_is_capital = data.groupby('is_capital').agg({"ARPU": 'var', "is_capital": lambda x: x.value_counts()/data.shape[0]})
data_stratified_is_capital.rename(columns={"is_capital": "stratum_share"}, inplace=True)
data_stratified_is_capital

,ARPU,stratum_share
is_capital,,
capital,11374.109085,0.322577
region,3130.864636,0.677423


In [ ]:
# посчитаем взвешенные дисперсии и просуммируем их
data_stratified_is_capital["ARPU_var_weighted"] = data_stratified_is_capital.ARPU * data_stratified_is_capital.stratum_share
strat_var = data_stratified_is_capital.ARPU_var_weighted.sum().round()

In [ ]:
print("Значение стратифицированной дисперсии для страты is_capital = ", strat_var)

Значение стратифицированной дисперсии для страты is_capital =  5790.0


## Задание 2 (2/3) ##
Рассчитайте значение стратифицированной дисперсии для страты is_pro

In [ ]:
data_stratified_is_pro = data.groupby('is_pro').agg({"ARPU": 'var', "is_pro": lambda x: x.value_counts()/data.shape[0]})
data_stratified_is_pro.rename(columns={"is_pro": "stratum_share"}, inplace=True)
data_stratified_is_pro

,ARPU,stratum_share
is_pro,,
0,4887.218947,0.948144
1,30086.317693,0.051856


In [ ]:
# посчитаем взвешенные дисперсии и просуммируем их
data_stratified_is_pro["ARPU_var_weighted"] = data_stratified_is_pro.ARPU * data_stratified_is_pro.stratum_share
strat_var = data_stratified_is_pro.ARPU_var_weighted.sum().round()

In [ ]:
print("Значение стратифицированной дисперсии для страты is_pro = ", strat_var)

Значение стратифицированной дисперсии для страты is_pro =  6194.0


## Задание 3 ##
Используя наилучшую страту, примените постстратификацию к данным и пересчитайте результаты t-test.

Введите значение p-value в поле ниже с точностью до трех знаков в десятичной части.

In [ ]:
gr_1 = data.query('group == 1')
gr_0 = data.query('group == 0')
strat_name = 'is_capital'
value_name = 'ARPU'
strat_weights = np.array(data.groupby(strat_name)[value_name].count()/data.shape[0])

In [ ]:
def get_strat_mean_var(data: pd.DataFrame, strat_name: str, value_name: str, population_strat_weights=None) -> dict:
    # calculates stratified mean and stratified variance:
    # data: pandas DataFrame
    # strat_name: name of the stratum column
    # value_name: name of the value column
    # population_strat_weights: stratum_share in population

    if population_strat_weights is None:
        population_strat_weights =  np.array(data.groupby(strat_name)[value_name].count()/data.shape[0])
    data_stratified = data.groupby(strat_name, as_index=False).agg({value_name: ['mean', 'var']})
    data_stratified.columns = [j[1] for j in data_stratified.columns.to_flat_index()]
    data_stratified['stratum_share'] = population_strat_weights

    data_stratified['strat_mean'] = data_stratified['mean'] * data_stratified['stratum_share']
    data_stratified['strat_var'] = data_stratified['var'] * data_stratified['stratum_share']

    result = {"strat_mean": data_stratified['strat_mean'].sum(), "strat_var": data_stratified['strat_var'].sum()}
    return result

In [ ]:
def strat_ttest(df_A: pd.DataFrame, df_B: pd.DataFrame, strat_column: str, value_name: str, population_strat_weights: np.array) -> dict:
  # df_A: pandas DataFrame test group 1
  # df_B: pandas DataFrame test group 2
  # strat_column: name of the stratum column
  # value_name: name of the target value column
  # population_strat_weights: stratum_share in population

  df_A_data = get_strat_mean_var(df_A, strat_column, value_name, population_strat_weights)
  df_B_data = get_strat_mean_var(df_B, strat_column, value_name, population_strat_weights)

  delta_mean_strat  = df_B_data['strat_mean'] - df_A_data['strat_mean']
  std_mean_strat = np.sqrt(df_A_data['strat_var'] / df_A.shape[0] + df_B_data['strat_var'] / df_B.shape[0])
  t_stat_strat = delta_mean_strat / std_mean_strat
  p_value = 2 * (t.sf(np.abs(t_stat_strat), df=len(df_A)+len(df_B)-2))

  return {'t_stat': t_stat_strat, 'p_value':p_value}

In [ ]:
control_df = data[data['group'] == 0]
test_df = data[data['group'] == 1]
strat_ttest(control_df, test_df, strat_name, value_name, strat_weights)

{'t_stat': 2.357821319048786, 'p_value': 0.018402283523607944}